In [11]:
import boto3
import sagemaker
from sagemaker.session import Session

role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

boto_session = boto3.Session(region_name=region)

default_bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker-featurestore'
offline_feature_store_bucket = 's3://mlops-rdi-sage791092-sagemaker-feature-store-bucket'

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

In [12]:
fg_list = sagemaker_client.list_feature_groups()
fg_list

{'FeatureGroupSummaries': [{'FeatureGroupName': 'mlops-rdi-sage791092-agg-feature-group',
   'FeatureGroupArn': 'arn:aws:sagemaker:eu-west-1:645143808269:feature-group/mlops-rdi-sage791092-agg-feature-group',
   'CreationTime': datetime.datetime(2022, 12, 16, 19, 20, 6, 844000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}}],
 'ResponseMetadata': {'RequestId': '5815a3e6-bdca-483b-bee0-e4fac8998877',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5815a3e6-bdca-483b-bee0-e4fac8998877',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '307',
   'date': 'Fri, 16 Dec 2022 21:16:32 GMT'},
  'RetryAttempts': 0}}

In [13]:
transaction_feature_group_name = fg_list['FeatureGroupSummaries'][0]['FeatureGroupName']
transaction_feature_group_name

'mlops-rdi-sage791092-agg-feature-group'

In [14]:
response = featurestore_runtime.get_record(
    FeatureGroupName=transaction_feature_group_name,
    RecordIdentifierValueAsString='2022-12-16 21:13:00.0',
    FeatureNames=[
        'tx_hour', 'total_nb_trx_1h', 'total_fee_1h', 'avg_fee_1h'
    ]
)
response

ValidationError: An error occurred (ValidationError) when calling the GetRecord operation: Validation Error: Attempted to parse the feature value for the feature named [tx_time] into a FeatureValue of type Fractional. The provided value must be within the range of a double precision floating point number defined by the IEEE 754 standard. The input format can be in either decimal form or scientific notation.